In [ ]:
import pandas as pd
    import numpy as np
    import featuretools as ft

    # Read in the data
    clients = pd.read_csv('C:/Users/ksmin/data_Science/dataset/clients.csv', parse_dates = ['joined'])
    loans = pd.read_csv('C:/Users/ksmin/data_Science/dataset/loans.csv', parse_dates = ['loan_start', 'loan_end'])
    payments = pd.read_csv('C:/Users/ksmin/data_Science/dataset/payments.csv', parse_dates = ['payment_date'])

    #Create entity set
    es = ft.EntitySet(id = 'clients')

     # Create an entity from the client dataframe
     # This dataframe already has an index and a time index
     # index=RDB primary key
     # time index= date_time feature
    #client entity and add es entityset
    es = es.entity_from_dataframe(entity_id = 'clients', 
                                   dataframe = clients, 
                                   index = 'client_id', 
                                   time_index = 'joined')
    #create entity loans
    es = es.entity_from_dataframe(entity_id='loans', 
                                   dataframe=loans,
                                   index='loan_id', 
                                   time_index='loan_end')
    # Create an entity from the payments dataframe
     # This does not have an index
    es = es.entity_from_dataframe(entity_id = 'payments', 
                                   dataframe = payments,
                                   variable_types = {'missed':
                                                     ft.variable_types.Categorical},
                                   make_index = True,
                                   index = 'payment_id',
                                   time_index = 'payment_date')

    # Group loans by client id and calculate mean, max, min of loans
    #client_id로 groupby를 한 다음 loan_amount의 mean,max,min 계산을 하여 출력한다.
    stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 
    'max', 'min'])
    #columns 생성한다.
    stats.columns = ['mean_loan_amount', 'max_loan_amount', 
    'min_loan_amount']
    # Merge with the clients dataframe
    stats = clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left')
    stats.head(10)
    
     #make entity realations
    # Create a relationship between clients and loans
    r_client_previous = ft.Relationship(es['clients']['client_id'],
                                        es['loans']['client_id'])
    # Add the relationship to the entity set
    es= es.add_relationship(r_client_previous)
    # Relationship between previous loans and previous payments
    r_payments = ft.Relationship(es['loans']['loan_id'],
                                 es['payments']['loan_id'])
    # Add the relationship to the entity set
    es= es.add_relationship(r_payments)

    # Create new features using specified primitives
    features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                     agg_primitives = ['sum'],
                                     trans_primitives=[])